This Markdown objective is to find the rent from the df column named infos. this column has many problems namely the enter is not the same. 

In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", sep = ';')
df

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,openinghours,interneturl,appointmenturl,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions
0,66,Cité Jeanne d'Arc,"À proximité de Sciences Po, de l'école de comm...",Poitiers Centre Ville,"La cité se situe à proximité de Sciences Po, d...",Salle TV\n Cuisine\n wifi,Cité Jeanne d'Arc\r\n 49 rue de la Cathédrale8...,49 rue de la Cathédrale 86000 POITIERS,cite.rochedargent@crous-poitiers.fr,05.49.11.97.78,NaN,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr,http://www.stockcrous.fr/visites/Jeanne/cite-j...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.5820732117,0.3445160091",http://www.stockcrous.fr/visites/Jeanne/cite-j...,Nouvelle-Aquitaine
1,73,Résidence Coureilles,À proximité de l'UFR de Droit et Gestion et de...,La Rochelle,La résidence se situe à proximité de l'UFR de ...,Salle de travail\n Internet\n Laverie\n Parkin...,Résidence Coureilles\r\n 170 rue de Coureilles...,170 rue de Coureilles 17000 LA ROCHELLE,logement.larochelle@crous-poitiers.fr,05.46.28.21.30,NaN,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr,http://www.stockcrous.fr/visites/Coureilles/re...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.1395797729,-1.1512320042",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine
2,75,Résidence Gémini,À côté de l'Ensma et à 5 min en transport du LPI.,Futuroscope,La résidence se situe à côté de l'Ensma et à 5...,Laverie\n Parking fermé\n Garage à vélos\n Asc...,Résidence Gémini\r\n 8 avenue Blaise PascalBP ...,8 avenue Blaise Pascal - BP 90176 - 86962 FUTU...,residence.gemini@crous-poitiers.fr,05.49.49.85.02,NaN,NaN,NaN,http://www.stockcrous.fr/visites/Gemini/reside...,NaN,NaN,"46.662399292,0.3600699902",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine
3,76,Résidence Jean Jouzel,À proximité de l'UFR de Droit et Gestionet de ...,La Rochelle,La résidence se situe à proximité de l'UFR de ...,Laverie\n Garage à vélos\n Parking\n Accessibi...,Résidence Jean Jouzel\r\n 172 rue de Coureille...,172 rue de Coureilles 17000 LA ROCHELLE cedex,logement.larochelle@crous-poitiers.fr,05.46.28.21.30,NaN,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr,http://www.stockcrous.fr/visites/Jouzel/reside...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.1401100159,-1.1510382891",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine
4,77,Résidence Jules Caisso,À proximité de l'UFR de Médecine-Pharmacie et ...,Poitiers Campus,La résidence se situe à proximité de l'UFR de ...,Laverie\n Parking\n Internet\n Accessibilité\n...,Résidence Jules Caisso\r\n 10-12 rue Marcel Do...,10-12 rue Marcel Doré - Bâtiment C21 86000 POI...,residence.julescaisso@crous-poitiers.fr,05.49.44.52.31,NaN,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr,http://www.stockcrous.fr/visites/Caisso/cite-j...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.565990448,0.3905799985",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,155,Parc Bellevue,NaN,Toulouse sud-est,Voir la visite en grand\n\n \n\n\n\nVoir d'aut...,Accès sécurisé\n Internet filaire + accès Eduo...,Parc Bellevue\r\n 3 avenue du professeur Josep...,3 avenue du professeur Joseph Ducuing 31400 To...,residence.parcbellevue@crous-toulouse.fr,05 62 25 62 75,NaN,http://www.crous-toulouse.fr/,http://www.crous-toulouse.fr/,NaN,http://www.crous-toulouse.fr/,http://www.crous-toulouse.fr/,"43.5606002808,1.4583499432",https://tourmkr.com/F1nVluNy8R,Occitanie
784,172,INSA,NaN,Toulouse sud-est,Voir la visite en grand\n\n \n\n\n\nVoir d'aut...,Accès sécurisé\n Veilleur de nuit\n Internet f...,Résidence INSA\r\n 135 avenue de Rangueil31077...,135 avenue de Rangueil 31077 Toulouse,hebergement.insa

Part I : Initialisation pour détecter la présence de symbole dans les mots. 

On va créer une fonction qui prend en argument un mot et une liste qui va déctecter s il y a un des élèments de cette liste dans le mot. Cette fonction va ensuite dans notre fonction principale. 

In [4]:
"""
String_checking est une fonction qui prend un mot en argument et un élèment et retourne 0 si l'élèmenet n'est 
pas inclus dans le mot et 1 inversement. 
"""


def string_checking(string, check):
    result = 0 
    if string.find(check) == -1:
        result = 0
    else: 
        result = 1
    return result


In [5]:
"""
Cette fonction prend en argument un mot à tester et une liste. Elle va utiliser la fonction précédente pour
vérifier qu'il n'y a aucun élèment de la liste dans notre mot. De même elle retourne 0 ou 1 en fonction de
s'il y a un élèment ou non. 
"""

def word_ban(string, liste):
    result = 0 
    for check in liste: 
        if string_checking(string, check) == 1: 
            result = 1 
        else: 
            result = 0
    return result 
            

Part II : Main functions 

README 

On va créer une fonction pour trouver le loyer à partir de la fonction précèdente. La fonction prend pour argument un texte (ici df['infos'][j] qui correspond au texte informatif d'une résidence où se trouve le loyer. La fonction va nettoyer le texte d'une manière très précise : 

1 - On va enlever tout les espaces du texte de façon à n'avoir qu'un block uniforme de texte. 
2 - on va enlever les parenthèses (problèmes souvent eu - à voir si c'est toujours le cas sans) 
3 - On va enlever les petits 2 des mètres carrés qui posent souvent dans la suite du code. 
4 - On va remplacer les à en € de façon à avoir les loyers qui sont inscrits de la manière suivante : 200 à 350€ (notrer recherche se fonde grâce au €). 
5 - On va remplacer les 'euros' en €
6 - on va remplacer les €€ en € car il peut y avoir des '200€à350€' qui deviennent '200€€350€'et nous on veut '200€350€'. 
7 - On va rajouter un espace après chaque '€'. 
8 - On normalise les , en . pour avoir les mêmes typologies
9 - on enlever les : (à voir si sa fonctionne toujours sans). 

On va ensuite tokeniser notre block de texte grâce à la fonction word_tokenize du package nltk. On va ainsi passer de blocks de caractères séparé par un espace à une liste de mots. 

Exemple (df["infos"][0]): 

On passe de :

"Logements HLM

70 T1 : 325 euros
14 T1bis de 30m² : 373,50 euros
13 T2 de 37m² : 404 euros

   
   
   
 Photos de la résidence"
   
à : 

"['LogementsHLM', '70T1', '325€', '14T1bisde30m373.50€', '13T2de37m404€', 'Photosdelarésidence', 'https//www.flickr.com/photos/crousaixmarseilleavignon/albums/72157679829208245']"

L'idée va ensuite de trouver le loyer grâce à cette liste de mot. On va ainsi appliquer la fonction rent. cette fonction prend en argument cette liste de mots. Pour chaque élèments de notre liste on va d'abord chercher l'index de € (s'il existe et s'il n'appartient pas à la liste des mots interdits - on ne veut pas les moments des cautions par exemple). On va ensuite regarder si le caractère à la gauche du € est un chiffre ou non. Si c'est un chiffre on va réetirer cela pour le caractère à gauche de ce dernier. Ainsi de suite jusqu'à trouver un élèment qui n'est pas un chiffre. Dans ce cas là on va arrêter et prendre le nombre entre les deux indices. Le problème est celui des virgule : on rajoute une condition qui dit que si on trouve une virgule on regarde à gauche si c'est un chiffre. On obtient ainsi une liste des loyers avec quelques valeurs abberrantes que l'on gera après. 



In [6]:
"""
Liste servant à ne pas prendre le montant des APL ou des cautions. Voir dans la suite du code. 
"""

List_banned_word = ['APL', 'Apl', 'Caution', 'caution', 'apl', 'CAUTION', "Charge", "de", "charge", "Charges", "charge", "Dépôtdegarantie","wifi", "wifirst","abonnement", "Logementssoumis", "machineàlaver"]


In [8]:
def rent(word):
    Loyer = [] #On crée une liste qui stockera le loyer
    #try:
    for i in range(0,len(word)): #on regarde dans chaque block de caractère 
        index = word[i].find("€") #on cherche l'index de l'euros normalement pour tout les loyers il y est 
        search = 0 #il nous servira à arrêter la boucle while 
        if word_ban(word[i],List_banned_word) == 0 and word[i].find("€") != -1: #On regarde si dans le block de mot il n'y a pas un mot banni comme APL pour éviter d'avoir le montant de ceux-ci, ce qui fausserait nos résultats.
            while search == 0: #Tant que notre variable de contrôle est égale à 0, on va chercher le loyer (ie : tant que ce sont des chiffres, on va chercher la nature (chffre ou non) de ce qui se situe à gauche.
                search_index = word[i][index - 1] #On cherche l'index de ce qui se trouve à gauche de l'euros
                try: #Try permet de tester si c'est un chiffre tout en ne bloquant pas les opérations avec une erreure
                    search_index = int(search_index) #On regare si à gauche de l'ancien index c'est un entier ou non (Dans le cas contraire on sera dans except).
                    index = index - 1 #on remonte de 1 l'index pour voir si ce qui se trouve à gauche de notre chiffre est également un chiffre.
                except: #si ce n'est pas un entier à gauche de l'euros ou du dernier chiffre identifié. 
                    if word[i][index - 1] == ".": #on regarde si ce n'est pas un point comme dans 350.85 euros. Si on ne met pas cette ligne on aurait seulement 85 et non 350.85. 
                        index = index - 2 #On va regarde ce qui est avant le point pour trouver le reste du montant
                        search_index = word[i][index - 1] #Sysiphe remonte sa pierre et recommence le même processus qu'avant
                        try: #on regarde de nouveau si à gauche du point c'est un entier
                            search_index = int(search_index) 
                            index = index - 1 
                        except:
                            search = 1 #si ce n'est pas un chiffre ie une lettre on arrête
                            index = -1 #permet de ne pas avoir des erreurs du style .50
    
                    else: #si ce n'est pas un point on arrête le compte. 
                        search = 1
                #print(word[i][index : word[i].find("euros")])

            if word[i][index : word[i].find("€")]!= '': #on regarde le fait que le token ne soit pas juste un '€'
                Loyer.append(float(word[i][index : word[i].find("€")])) #On ajoute au loyer le loyer trouvé
    #except:
        #if word[i] != "€" :
            #List_banned_word.append(word[i])
            #print(List_banned_word)
            #rent(word)
    return Loyer

In [12]:
def rent_find(residence): 
    residence = residence.replace(' ', '')
    residence = residence.replace('(', '')
    residence = residence.replace(')', '')
    residence = residence.replace('²', '')
    residence = residence.replace('mà', '')
    residence = residence.replace('à', '€')
    residence = residence.replace('euros', '€')
    residence = residence.replace('€€', '€')
    residence = residence.replace('€', '€ ')
    residence = residence.replace(',', '.')
    word =  nltk.word_tokenize(residence, language='french') #on transforme notre scripts en block de mots. ceux qui nous intéressent ont un euros à la fin (mais il y en a qui ont un euros à la fin et qui ne nous intéresse pas
    return rent(word)
    

Part III : construction du Dataframe 

on crée une boucle for sur toute les résidences. si la fonction foctionne où l'ajoute à une liste sinon on ajoute NAN et on crée la colone loyer. 

In [13]:
a = 0
b = 0
Loyer = []
for j in range(0,df.shape[0]):
    try:
        Loyer.append(rent_find(df["infos"][j]))
        b = b + 1
    except: 
        Loyer.append("NaN")
        a = a + 1
df['Loyer'] = Loyer
print("échec pour ",a)
print("réussite pour ", b)
print("total ", a + b)

échec pour  11
réussite pour  777
total  788


In [26]:
"""
fonction servant à ne garder que les loyers et enlever les quelques erreures du style 5.99 qui serait l'abonnement au wifi
"""

def remove_fun(residence):
    for i in range(0, len(residence)):
        integer = int(residence[i]) #on transforme le loyer i de la résidence en un entier 
                                #pour ne pas avoir les chiffres derrières la virgule
        text = str(integer) #on transforme cet entier en texte
        if len(text) != 3: 
            residence.remove(residence[i]) #on enlève dès que c'est différent de 3 ie ce n'est pas un 
                                           #loyer qui pourrait être au Crous.                          
    return residence

    
               

In [28]:
"""
A lancer plusieurs fois - la fonction len à l'air de buguer sur python
"""

for j in range(0,df.shape[0]):
    if df["Loyer"][j] != "NaN": #s'il y a un Loyer
        try : 
            remove_fun(df["Loyer"][j]) #on texte la fonction 
        except:  #On ressaye ainsi de suite
            try: 
                remove_fun(df["Loyer"][j])
            except: 
                remove_fun(df["Loyer"][j])
                try: 
                    remove_fun(df["Loyer"][j])
                except: 
                    remove_fun(df["Loyer"][j])

In [30]:
"""
On crée trois colonnes stockant le loyer maximum, le loyer minimum et la moyenne des loyers. 
"""

Max = []
Min = []
Mean = []
a = 0
for j in range(0,df.shape[0]):
    if df["Loyer"][j] != "NaN" and df["Loyer"][j] != []:
        a = a + 1
        Max.append(max(df["Loyer"][j]))
        Min.append(min(df["Loyer"][j]))
        Mean.append(np.mean(df["Loyer"][j]))
    else: 
         Max.append("NaN")
         Min.append("NaN")
         Mean.append("NaN")
df['Max Loyer'] = Max
df['Min Loyer'] = Min
df['Mean Loyer'] =  Mean
print(a)

481


In [43]:
df.head(20)

,id,title,short_desc,zone,infos,services,contact,address,mail,phone,...,virtualvisiturl,bookingurl,troubleshootingurl,geocalisation,photo,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer
0,66,Cité Jeanne d'Arc,"À proximité de Sciences Po, de l'école de comm...",Poitiers Centre Ville,"La cité se situe à proximité de Sciences Po, d...",Salle TV\n Cuisine\n wifi,Cité Jeanne d'Arc\r\n 49 rue de la Cathédrale8...,49 rue de la Cathédrale 86000 POITIERS,cite.rochedargent@crous-poitiers.fr,05.49.11.97.78,...,http://www.stockcrous.fr/visites/Jeanne/cite-j...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.5820732117,0.3445160091",http://www.stockcrous.fr/visites/Jeanne/cite-j...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
1,73,Résidence Coureilles,À proximité de l'UFR de Droit et Gestion et de...,La Rochelle,La résidence se situe à proximité de l'UFR de ...,Salle de travail\n Internet\n Laverie\n Parkin...,Résidence Coureilles\r\n 170 rue de Coureilles...,170 rue de Coureilles 17000 LA ROCHELLE,logement.larochelle@crous-poitiers.fr,05.46.28.21.30,...,http://www.stockcrous.fr/visites/Coureilles/re...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.1395797729,-1.1512320042",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
2,75,Résidence Gémini,À côté de l'Ensma et à 5 min en transport du LPI.,Futuroscope,La résidence se situe à côté de l'Ensma et à 5...,Laverie\n Parking fermé\n Garage à vélos\n Asc...,Résidence Gémini\r\n 8 avenue Blaise PascalBP ...,8 avenue Blaise Pascal - BP 90176 - 86962 FUTU...,residence.gemini@crous-poitiers.fr,05.49.49.85.02,...,http://www.stockcrous.fr/visites/Gemini/reside...,NaN,NaN,"46.662399292,0.3600699902",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
3,76,Résidence Jean Jouzel,À proximité de l'UFR de Droit et Gestionet de ...,La Rochelle,La résidence se situe à proximité de l'UFR de ...,Laverie\n Garage à vélos\n Parking\n Accessibi...,Résidence Jean Jouzel\r\n 172 rue de Coureille...,172 rue de Coureilles 17000 LA ROCHELLE cedex,logement.larochelle@crous-poitiers.fr,05.46.28.21.30,...,http://www.stockcrous.fr/visites/Jouzel/reside...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.1401100159,-1.1510382891",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
4,77,Résidence Jules Caisso,À proximité de l'UFR de Médecine-Pharmacie et ...,Poitiers Campus,La résidence se situe à proximité de l'UFR de ...,Laverie\n Parking\n Internet\n Accessibilité\n...,Résidence Jules Caisso\r\n 10-12 rue Marcel Do...,10-12 rue Marcel Doré - Bâtiment C21 86000 POI...,residence.julescaisso@crous-poitiers.fr,05.49.44.52.31,...,http://www.stockcrous.fr/visites/Caisso/cite-j...,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"46.565990448,0.3905799985",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
5,83,Résidence L'auberge Espagnole,Sur le campus de l'image,Angoulême,Résidence réservée aux étudiants du pôle Image...,Laverie\n Wifi\n Accessibilité\n Parking\n Gar...,Résidence L'auberge Espagnole\r\n 14 rue Saint...,14 rue Saint Cybard 16000 ANGOULEME,residences.angouleme@crous-poitiers.fr,05.45.25.51.51,...,NaN,https://www.crous-poitiers.fr,https://www.crous-poitiers.fr/logements/demand...,"45.6562576294,0.1484660059",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
6,84,Résidence Henri-Georges Clouzot,Sur le pôle universitaire,Niort,La résidence se situe sur le pôle universitair...,Laverie\n Internet\n Accessibilité\n Parking f...,Résidence Henri-Georges Clouzot\r\n 7 rue du G...,NaN,NaN,NaN,...,NaN,NaN,NaN,"46.3264083862,-0.4883550107",https://admin-v2.crous-mobile.fr//media/crous-...,Nouvelle-Aquitaine,[],NaN,NaN,NaN
7,1189,Résidence Aziyadé,à proximité de l'UFR de Droit/Gestion et de l'IUT,La Rochelle,La résidence se situe à prox